## Генераторы

В прошлый раз мы обсуждали **iterable** и **iterators**. На примере класса MyRange, мы видели что для реализации логики перечисления элементов объекта нужно писать довольно-таки много кода:

```
class MyIterator:
    def __init__(self, obj):
        self.start = obj.start
        self.end = obj.end
        self.step = obj.step
    def __next__(self):
        if self.start < self.end:
            ret = self.start
            self.start += self.step
            return ret
        else:
            raise StopIteration
class MyRange:
    def __init__(self, start, end, step=1):
        self.start = start
        self.end = end
        self.step = step
    def __iter__(self):
        return MyIterator(self)
```

Ту же самую логику можно имплементировать с помощью **генераторов**. Но перед этим давайте разберемся с ключевым словом **yield**.

In [8]:
import time

def Hello():
    yield f'current time is {time.strftime("%H:%M:%S %d.%m.%Y")}'
    yield 'Yield #2'  

In [9]:
generator = Hello()
print(type(generator))

<class 'generator'>


In [10]:
print(next(generator))
print(next(generator))

current time is 18:18:41 30.11.2022
Yield #2


In [11]:
print(next(generator))

StopIteration: 

In [12]:
'__iter__' in dir(generator)

True

In [13]:
'__next__' in dir(generator)

True

Таким образом, наличие в функции команды **yield** создает нам генератор. Давайте поисследуем поведение ее поведение:

In [16]:
def f():
    print('перед первым yield')
    yield 'Привет!!!'
    print('что-то между yield-ами')
    yield 'Пока!!!'
    print('после последнего yield')
    
generator = f()

In [17]:
for x in generator:
    print('I AM INSIDE FOR LOOP:', x)

перед первым yield
I AM INSIDE FOR LOOP: Привет!!!
что-то между yield-ами
I AM INSIDE FOR LOOP: Пока!!!
после последнего yield


Получается что когда интепретатор доходит до **yeild** он возвращает то на что указывает **yeild** и затем останавливает исполнение функции. Если позвать функцию заново - мы продолжим с момента на котором остановились, т.е. мы запомнили предыдущее состояние.

In [23]:
def f():
    print('перед первым yield')
    yield 'Привет!!!'
    return 'я return!!!!'
    yield 'Пока!!!'
    print('после последнего yield')
    
generator = f()

In [22]:
for x in generator:
    print(x)

перед первым yield
Привет!!!


In [24]:
print(next(generator))
print(next(generator))

перед первым yield
Привет!!!


StopIteration: я return!!!!

Упражнения:

1. Напишите генератор my_range
2. Генератор бесконечно генерирующий простые числа

In [49]:
def my_range(start, end, step):
    end, start = start, end
    while start > end:
        yield start
        start -= step

In [50]:
r = my_range(0, 100, 10)
for x in r:
    print(x, end=' ')

100 90 80 70 60 50 40 30 20 10 

In [26]:
r = my_range(0, 100, 10)
for x in r:
    print(x, end=' ')

0 10 20 30 40 50 60 70 80 90 

In [31]:
def primes():
    p = 2
    yield p
    while True:
        p += 1
        flag = True
        for i in range(2, int(p**0.5)+1):
            if p % i == 0:
                flag = False
                break
        if flag is True:
            yield p

In [32]:
p = primes()

In [33]:
for i, x in enumerate(p):
    print(x, end=' ')
    if i > 9:
        break

2 3 5 7 11 13 17 19 23 29 31 

Если у **list comprehension** заменить скобки на круглые, то вернется не кортеж, а генератор:

In [34]:
[x*x for x in range(1, 11)]

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [35]:
(x*x for x in range(1, 11))

<generator object <genexpr> at 0x7ff3e04df0f8>

In [36]:
g = (x*x for x in range(1, 11))
for x in g:
    print(x, end=' ')

1 4 9 16 25 36 49 64 81 100 

Конструктор list() отлично справляется с генераторами и итераторами:

In [37]:
g = (x*x for x in range(1, 11))

list(g) # и другие iterable конструктора типа tuple, set и тд

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [40]:
set(my_range(0, 100, 10))

{0, 10, 20, 30, 40, 50, 60, 70, 80, 90}

In [41]:
r = my_range(0, 100, 10)

In [43]:
list(my_range(0, 100, 10))[::-1]

[90, 80, 70, 60, 50, 40, 30, 20, 10, 0]

In [42]:
for x in r[::-1]:
    print(x, end=' ')

TypeError: 'generator' object is not subscriptable

## Множественное наследование. MRO. Super

Python поддерживает множественное наследование. Синтаксис очень прост:

In [53]:
class B:
    att_b = 'I am the attribute of B!'

class C:
    att_c = 'I am the attribute of C!'

class A(B, C):
    pass

In [54]:
a = A()
print(a.att_b, a.att_c, sep='\n')

I am the attribute of B!
I am the attribute of C!


Упражнение: Adress, Person - соединить в один с помощью множественного наследования.

In [55]:
class Address:
    def __init__(self, street, city):
        self.street = str(street)
        self.city = str(city)
        
    def show(self):
        print(self.street)
        print(self.city)

class Person:
    def __init__(self, name, email):
        self.name = name
        self.email= email
        
    def show(self):
        print(self.name + ' ' + self.email)

In [56]:
class PersonAndAdress(Person, Address):
    def __init__(self, name, email, street, city):
        Person.__init__(self, name, email)
        Address.__init__(self, street, city)

In [57]:
p = PersonAndAdress('Ivan', 'abc@gmail.com', 'Пушкина', 'Москва')

In [58]:
p.name, p.email, p.street, p.city

('Ivan', 'abc@gmail.com', 'Пушкина', 'Москва')

In [59]:
class PersonAndAdress(Person, Address):
    def __init__(self, name, email, street, city):
        super().__init__(name, email)
        super(Person, self).__init__(street, city)
        
# PersonAndAdress  ----> Person -----> Address

In [60]:
p = PersonAndAdress('Ivan', 'abc@gmail.com', 'Пушкина', 'Москва')

In [61]:
p.name, p.email, p.street, p.city

('Ivan', 'abc@gmail.com', 'Пушкина', 'Москва')

Возможность создавать множественные наследования может вести к довольно громоздким и запутанным графам отношений между классами. В таком случае, если у текущего класса есть несколько предков, у которых аттрибуты имеют одинаковые имена, непонятно в каком порядке этот аттрибут ищется.

Например:

![ex](https://i.imgur.com/Eemp2jY.png)

In [64]:
class O:
    pass
class A(O):
    pass
class B(O):
    pass
class C(O):
    pass
class D(O):
    pass
class E(O):
    pass

class K1(A, B, C):
    pass
class K2(B, D):
    pass
class K3(C, D, E):
    pass

class Z(K1, K2, K3):
    pass

Команда **mro (METHOD RESOLUTION ORDER)** выводит порядок разрешения методов:

In [66]:
def print_mro(T):
    print(*[c.__name__ for c in T.mro()], sep=' -> ')

In [67]:
print_mro(Z)

Z -> K1 -> A -> K2 -> B -> K3 -> C -> D -> E -> O -> object


В каком порядке?

Ответ: алгоритм **C3 linearization**. О нем можно почитать здесь:

- https://en.wikipedia.org/wiki/C3_linearization
- https://dl.acm.org/doi/pdf/10.1145/236337.236343

Важно понимать что сохраняется здравый смысл:
- если имеем ```class A(B, C, D)```, то в MRO однозначно имеем что A идет до В, В до C и С до D (возможно между ними что-то есть)
- родители идут до grandparents и т.д.
- object всегда последний элемент

In [69]:
class X:
    pass
class Y:
    pass
class A(X, Y):
    pass
class B(Y, X):
    pass
class G(A, B):
    pass

# Что произошло? Придумайте еще похожий пример

TypeError: Cannot create a consistent method resolution
order (MRO) for bases X, Y

## Магические методы и перегрузка операторов

Магические методы - особые методы, имя которых начинается и заканчивается двойным подчеркиванием. Они не предназначены для вызова напрямую, эти методы вызываются внутренним образом при каком-то действии.

Например ``` a + b ``` эквивалентно ```a.__add__(b)```

In [70]:
a = 1
b = 2
a.__mul__(b) == a*b

True

С большим числом таких методов мы уже знакомы (```__init__```, ```__dir__``` и тд). Давайте познакомимся с методами которые определяют поведение операторов для данного класса:

![operators1](https://i.imgur.com/ykYFkRY.png)

![operators2](https://i.imgur.com/SCUqZWU.png)

Задача:

Напишите класс матриц, перегрузите операции сложения, вычитания, умножения и строкового вывода.

P.s. а что для матриц является делением?

In [92]:
class Matrix:
    
    def __init__(self):
        n, m = map(int, input('Введите n и m: ').split())
        self.mat = []
        self.n = n
        self.m = m
        for i in range(n):
            self.mat.append(list(map(int, input(f'Введите строку {i+1}: ').split())))
            
    def __add__(self, other):
        ret = [[0 for _ in range(self.m)] for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                ret[i][j] += (self.mat[i][j] + other.mat[i][j])
        return ret
    
    def __mul__(self, other):
        ret = [[0 for _ in range(other.m)] for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                
                for k in range(self.m): # other.n
                    ret[i][j] += self.mat[i][k] * other.mat[k][j]

        return ret

In [93]:
A = Matrix()
B = Matrix()

Введите n и m: 2 2
Введите строку 1: 1 2
Введите строку 2: 3 4
Введите n и m: 2 2
Введите строку 1: 1 0
Введите строку 2: 0 1


In [90]:
A+B

[[2, 2], [3, 5]]

In [94]:
A*B

[[1, 2], [3, 4]]

In [107]:
A = Matrix()
B = Matrix()

In [108]:
print(A)

## @staticmethod

Встроенный декоратор для определения статичных методов - без превязки к классу или его экземпляра. Идея в том что этот метод логично связан с нашим классом, но ему не нужны ни класс, ни экземпляр.

## @classmethod

Встроенный декоратор для определения метода класса. В данном случае, первым аргументов идет не экземпляр **self**, а сам класс **cls**. Позволяет доставать аттрибуты класса и часто используется как альтернативный конструктор


Оба декоратора еще хороши тем, что само их наличие сразу говорит о том какой тип метода написан.

In [95]:
class A:
    def f():
        print("hi")

In [96]:
A.f()

hi


In [97]:
A().f()

TypeError: f() takes 0 positional arguments but 1 was given

In [98]:
class A:
    @staticmethod
    def f():
        print("hi")

In [99]:
A.f()
A().f()

hi
hi


In [100]:
class Date(object):
    
    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year

    @classmethod
    def from_string(cls, date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        date1 = cls(day, month, year)
        return date1

    @staticmethod
    def is_date_valid(date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        return day <= 31 and month <= 12 and year <= 3999

date2 = Date.from_string('30-11-2022')
is_date = Date.is_date_valid('30-11-2022')

In [101]:
date2.day, date2.month, date2.year

(30, 11, 2022)

In [ ]:
# A)
# B)
# C)

## Датаклассы

Рассмотрим случай, когда наш класс совсем прост - есть лишь поля (аттрибуты) экземпляров и нет никаких методов:

In [122]:
class Person:
    def __init__(self, name, surname, email):
        self.name = name
        self.surname = surname
        self.email = email

In [123]:
p = Person('Ivan', 'Ivanov', 'abc@gmail.com')

В таких случаях Python предлагает альтернативный синтаксис с помощью декоратора **@dataclass**:

In [125]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    surname: str
    email: str

In [126]:
p = Person('Ivan', 'Ivanov', 'abc@gmail.com')

In [127]:
p.name

'Ivan'

Можно задавать параметры в декораторе, например делать аттрибуты неизменяемыми

In [128]:
@dataclass(frozen=True)
class Person:
    name: str
    surname: str
    email: str

In [129]:
p = Person('Ivan', 'Ivanov', 'abc@gmail.com')

In [130]:
p.name = 'Oleg'

FrozenInstanceError: cannot assign to field 'name'